In [8]:
from itertools import islice 
import numpy as np
import pandas as pd
import math
import random
from sklearn.utils import shuffle

In [9]:
max_students = 33
test_list = pd.Series(range(1,max_students+1,1)).T

In [10]:
#Insert the minimum and maximum number of students that might enter
#We tried to do 5 times all numbers
min_num = 2 
max_num = 11
num_repetitions = 5

num_in_exp = pd.Series(range(min_num,max_num+1,1)).T 
random.shuffle(num_in_exp)
for j in range(0,num_repetitions-1):
    add = pd.Series(range(min_num,max_num+1,1)).T
    random.shuffle(add)
    num_in_exp = pd.concat([num_in_exp, add])

num_in_exp.reset_index(drop = True, inplace = True)

In [11]:
#Result is the dataframe that gives the sequences to the students where all rounds are below each other 
#Result_all is a dataframe where all rooms are below each other
result = pd.DataFrame() 
result_all = pd.DataFrame()

#The number of persons in each of the rooms 
#should add up to max_students 
#cannot be less than max_num
num_per_exp_A = 11
num_per_exp_B = 11
num_per_exp_C = 11

#num_same is the number of rounds students stay in the same room. 
#so after num_same experiments, all students change room
num_same = 5

#num_same_counter is the amount of times a student can get the same position. 
#This amount increases as the number of experiments increases by num_rounds_for_same
num_same_counter = 2
num_rounds_for_same = 6

#num_rearranged is the number of rounds that are already there (?)
num_rearranged = 0

#the_sample is the list of student numbers
the_sample = test_list

#We increase the length of the dataframe when certain conditions are satisfied. 
while len(result) < (max_num - min_num+1)*num_repetitions: 
    #If the number of rounds for which we have an order is larger than a multiple of num_rounds_for_same, we increase the counter. 
    if len(result) >= (num_same_counter-1) * num_rounds_for_same: 
        num_same_counter = num_same_counter+1
    #We shuffle all students every num_same rounds. 
    if len(result) >= (num_rearranged + 1)*num_same:
        the_sample = test_list.sample(max_students)
        num_rearranged = num_rearranged + 1
        
    #The temporary results WHY CALLED LIKE THIS? 
    result_temp = pd.DataFrame()
    result_all_temp = pd.DataFrame()
    
    #From our sample, we take the first num_per_exp_A students, and suffle those
    add_A = the_sample.iloc[0:num_per_exp_A].reset_index(drop = True).to_frame().T
    add_A = shuffle(add_A.T).reset_index(drop=True).T
    #add_A = pd.concat([add_A, pd.Series(np.NaN)], axis = 1)
    result_all_temp = pd.concat([result_all, add_A])
    
    #The same for the next num_per_exp_B students
    add_B = the_sample.iloc[num_per_exp_A:(num_per_exp_A+num_per_exp_B)].reset_index(drop = True).to_frame().T 
    add_B = shuffle(add_B.T).reset_index(drop=True).T
    result_all_temp = pd.concat([result_all_temp, add_B])
    
    #The same for the next num_per_exp_C students
    add_C = the_sample.iloc[(num_per_exp_A+num_per_exp_B):(num_per_exp_A+num_per_exp_B+num_per_exp_C)].reset_index(drop = True).to_frame().T 
    add_C = shuffle(add_C.T).reset_index(drop=True).T
    result_all_temp = pd.concat([result_all_temp, add_C])
    
    
    #Check if there is no position where someone is seated more than num_same_counter times
    #We only need to check until num_in_exp, the other positions do not matter.
    if sum([len([x for x,y in result_all_temp[j].value_counts().items() if y > num_same_counter]) > 0 for j in range(0,num_in_exp.iloc[len(result_temp)]+1)]) == 0:
        tot = pd.concat([add_A, add_B], axis = 1)
        tot = pd.concat([tot,add_C], axis = 1)
        result_temp = pd.concat([result, tot], axis = 0)  

        #In result_all_temp we set all not used positions to NaN
        for j in range(0,len(result)+1):
            for k in range(0,3):
                m = j*3+k
                for i in range(num_in_exp.iloc[j],max(num_per_exp_A, num_per_exp_B, num_per_exp_C)):
                    result_all_temp.iloc[m,i] = np.NaN

        #Maximum is the maximum number of times someone is at the same position
        maximum = 0
        for j in range(0,max(num_per_exp_A,num_per_exp_B,num_per_exp_C)):
            if (result_all_temp[j].isnull().sum() < len(result_all_temp)): 
                if max(result_all_temp[j].value_counts()) > maximum: 
                    maximum = max(result_all_temp[j].value_counts())

        #Maximum2 is the maximum number of times someone is at the same position while they are in the same room (num_same)
        maximum2 = 0
        for j in range(0,max(num_per_exp_A,num_per_exp_B,num_per_exp_C)):
            if (result_all_temp.iloc[num_rearranged*num_same*3:(num_rearranged+1)*num_same*3,:][j].isnull().sum() < len(result_all_temp) - num_rearranged*num_same*3): 
                if max(result_all_temp.iloc[num_rearranged*num_same*3:(num_rearranged+1)*num_same*3,:][j].value_counts()) > maximum2: 
                    maximum2 = max(result_all_temp.iloc[num_rearranged*num_same*3:(num_rearranged+1)*num_same*3,:][j].value_counts())

        #When the maximum number of times someone is at the same position does not exceed num_same_counter-1, 
        #and the maximum number of times someone is at the same position while they are in the same room (num_same) is not more than 1
        #then we accept the next sequence. 
        if ((maximum < num_same_counter) & (maximum2 < 2)):
            result = result_temp
            result_all = result_all_temp 
            print(len(result))
        del result_temp
        del result_all_temp 
        #Otherwise, we try again. 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [12]:
new_index = list(range(1,len(result)+1))
result.columns = [x+1 for x in list(result.columns)]
result.index = new_index
#result.to_csv("StudentOrderings_small/Student_orderings.csv", ',')

In [13]:
result_A = result.iloc[:,0:num_per_exp_A]
result_B = result.iloc[:,num_per_exp_A:(num_per_exp_A + num_per_exp_B)]
result_C = result.iloc[:,(num_per_exp_A + num_per_exp_B):(num_per_exp_A + num_per_exp_B+num_per_exp_C)]

In [14]:
#To ease the writing of the result during the experiment, we add a NaN where one should stop writing 
result_A_nec = result_A
for j in range(0,(max_num - min_num+1)*num_repetitions):
    result_A_temp = pd.DataFrame()
    result_A_temp = pd.concat([result_A_temp, result_A.iloc[j, :num_in_exp.iloc[j]]],axis = 1, ignore_index = True)
    if num_in_exp.iloc[j] < max(num_in_exp):
        result_A_temp = pd.concat([result_A_temp, pd.Series(np.NaN)], ignore_index = True)
    result_A_temp = pd.concat([result_A_temp, result_A.iloc[j, num_in_exp.iloc[j]:]],axis = 0, ignore_index =True)
    result_A_nec.iloc[j,:] = result_A_temp.T

result_B_nec = result_B
for j in range(0,(max_num - min_num+1)*num_repetitions):
    result_B_temp = pd.DataFrame()
    result_B_temp = pd.concat([result_B_temp, result_B.iloc[j, :num_in_exp.iloc[j]]],axis = 1, ignore_index = True)
    if num_in_exp.iloc[j] < max(num_in_exp):
        result_B_temp = pd.concat([result_B_temp, pd.Series(np.NaN)], ignore_index = True)
    result_B_temp = pd.concat([result_B_temp, result_B.iloc[j, num_in_exp.iloc[j]:]],axis = 0, ignore_index =True)
    result_B_nec.iloc[j,:] = result_B_temp.T
    
result_C_nec = result_C
for j in range(0,(max_num - min_num+1)*num_repetitions):
    result_C_temp = pd.DataFrame()
    result_C_temp = pd.concat([result_C_temp, result_C.iloc[j, :num_in_exp.iloc[j]]],axis = 1, ignore_index = True)
    if num_in_exp.iloc[j] < max(num_in_exp):
        result_C_temp = pd.concat([result_C_temp, pd.Series(np.NaN)], ignore_index = True)
    result_C_temp = pd.concat([result_C_temp, result_C.iloc[j, num_in_exp.iloc[j]:]],axis = 0, ignore_index =True)
    result_C_nec.iloc[j,:] = result_C_temp.T
    
#result_A_nec.to_csv("StudentOrderings_small/Student_orderings_A.csv", ',')
#result_B_nec.to_csv("StudentOrderings_small/Student_orderings_B.csv", ',')
#result_C_nec.to_csv("StudentOrderings_small/Student_orderings_C.csv", ',')

In [10]:
#num_in_exp.index= new_index
#num_in_exp.to_csv("StudentOrderings_small/Num_per_exp.csv", ',')

In [11]:
#To write the documents for each student 
#orderings = result
#for i in range(1,max_students+1):
#    with open('StudentOrderings_small/Student{}.txt'.format(i), 'w') as f: 
#        f.write("Student nummer {} van {}".format(i, max_students))
#        f.write('\n')
#        f.write('\n')
#        for j in range(0,len(orderings)):
#            z = 0
#            for k in range(0,num_per_exp_A+1):
#                if (math.isnan(orderings[orderings == i].iloc[j,k])) is not True: 
#                    f.write("Ronde {} - A".format(j+1))
#                    f.write('\n')
#                    z = 1
#            for k in range(num_per_exp_A+1,(1+num_per_exp_A+num_per_exp_B)):
#                if (math.isnan(orderings[orderings == i].iloc[j,k])) is not True: 
#                    f.write("Ronde {} - B".format(j+1)) 
#                    f.write('\n')
#                    z = 1
#            for k in range((num_per_exp_A+num_per_exp_B+1),(1+num_per_exp_A+num_per_exp_B+num_per_exp_C)):
#                if (math.isnan(orderings[orderings == i].iloc[j,k])) is not True: 
#                    f.write("Ronde {} - C".format(j+1))
#                    f.write('\n')
#                    z = 1
#                    
#            if z == 0: 
#                print("FOUT for j is {} and i is {}".format(j, i))
#                f.write("Ronde {} overslaan".format(j+1))
#                f.write('\n')

In [16]:
result_A_nec

,1,2,3,4,5,6,7,8,9,10,11
1,1,9,11.0,6.0,NaN,7.0,8.0,4.0,2.0,10.0,5.0
2,7,11,2.0,NaN,6.0,3.0,1.0,5.0,10.0,4.0,8.0
3,2,4,9.0,5.0,10.0,6.0,11.0,NaN,3.0,1.0,7.0
4,3,1,5.0,10.0,11.0,7.0,9.0,4.0,6.0,2.0,NaN
5,4,7,6.0,1.0,3.0,8.0,NaN,5.0,9.0,2.0,11.0
6,12,21,8.0,19.0,22.0,30.0,3.0,29.0,24.0,NaN,4.0
7,24,29,19.0,32.0,30.0,NaN,21.0,4.0,3.0,12.0,8.0
8,3,12,NaN,4.0,21.0,24.0,19.0,30.0,29.0,8.0,22.0
9,4,19,29.0,3.0,32.0,22.0,24.0,21.0,NaN,30.0,8.0
10,29,24,30.0,21.0,19.0,12.0,8.0,32.0,4.0,3.0,22.0


In [1]:
def splitk_geometric_mean_n_start2(p,k,nmax):
    #starts at customer 2
    sum_ = 0
    sum_2 = 0
    l_max = int(np.floor(np.log(nmax)/np.log(k)))
    sum_probs =0
    #print(l_max)
    for x in range(1,l_max+1):
        #print(x)
        for j in range(pow(k,x-1) +1, pow(k,x)+1):
            #print(j/pow(2,x))
            #sum_ = sum_ + j/pow(k,x)
            sum_ = sum_ + j*pow(1-p,j-2)*p/pow(k,x)
            sum_2 = sum_2 + (1-j/pow(k,x))*(1-j/pow(k,x))
            sum_probs = sum_probs + pow(1-p,j-2)*(p)
            
    for j in range(pow(k,l_max) + 1, nmax+1):
        sum_ = sum_ +  j*pow(1-p,j-2)*(p)/pow(k,l_max+1)
        sum_2 = sum_2 +  (1-j/pow(k,l_max+1))*(1-j/pow(k,l_max+1))
        sum_probs = sum_probs + pow(1-p,j-2)*(p)
        
    return (sum_probs-sum_, sum_probs)